## Feature Extraction by Pyradiomics

In [1]:
from __future__ import print_function
import os
import json
import collections
import SimpleITK as sitk
import numpy as np
import pandas as pd
from tqdm import tqdm

import six
import re
import radiomics
from radiomics import featureextractor, getFeatureClasses
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm

In [7]:
featureClasses = getFeatureClasses()
path = '/mnt/sda1/test2-set/nifti'
json_path = '/mnt/sda1/test-set/json'
params = 'param_extraction.yaml'
extractor = featureextractor.RadiomicsFeatureExtractor(params)

In [8]:
print('Enabled input images:')
for imageType in extractor.enabledImagetypes.keys():
    print('\t' + imageType)

Enabled input images:
	Original
	LoG
	Wavelet


### Feature Extraction

In [9]:
def ndarray2list(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: ndarray2list(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [ndarray2list(i) for i in obj]
    else:
        return obj

In [11]:
extracted = []
patients = os.listdir(path)
patients.sort()

for patient in tqdm(patients, desc="Processing patients"):
    imagePath = None
    maskPath = None
    folder = os.path.join(path,patient)
    output = os.path.join(json_path, f"{'axial' if patient.endswith('a') else 'coronal'}", f"{patient}.json")
    
    if os.path.isfile(output):
        continue
    
    if not(os.path.isdir(folder)):
        print(f'{patient} is not a folder')
        continue
    
    if (f"{patient}.nii.gz") in os.listdir(folder):
        imagePath = os.path.join(folder,f"{patient}.nii.gz")
    else: 
        if (f"{patient}.nii") in os.listdir(folder):
            imagePath = os.path.join(folder,f"{patient}.nii")
        
    if (f"RTSTRUCT {patient}.nii.gz") in os.listdir(folder):
        maskPath = os.path.join(folder,f"{patient}.nii.gz")
    else: 
        if (f"RTSTRUCT {patient}.nii") in os.listdir(folder):
            maskPath = os.path.join(folder,f"{patient}.nii")
        
    if imagePath is None or maskPath is None:
        print(f"Missing {patient} file")
        continue

    try:
        mask = sitk.ReadImage(maskPath)
        # print(f"Running {patient}")
        featureVector = extractor.execute(imagePath,mask)
    except:
        print(f"Error when extracting {patient} feature")
        continue
    feature = {
        "Name": patient,
        "Label": 1 if patient.split('_')[0]=='Infective' else 0,
        **featureVector
    }

    with open(output, 'w') as json_file:
        json.dump(ndarray2list(feature), json_file, indent = 4)

Processing patients: 100%|████████████████████████| 5/5 [08:12<00:00, 98.47s/it]
